In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import pandas as pd

In [62]:
# Set headers
heads = requests.utils.default_headers()
heads.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
   })

In [63]:
# helper functions to retrieve detials about each post
def getUser(header):
    user = ''
    user_tag = header.find("a", class_="user")
    if user_tag:
        user_tag.get_text()
    return user

def getTimestamp(header):
    time = ''
    date = ''
    tag_datetime = header.contents[-1]
    if tag_datetime:
        time = tag_datetime.contents[0].contents[0]
    if len(tag_datetime.contents) == 3:
        date = tag_datetime.contents[-1].contents[0]
    elif len(tag_datetime.contents) == 6:
        date = tag_datetime.contents[2].contents[0] + ' ' + tag_datetime.contents[4].contents[0]
    date_time = '{} {}'.format(date, time) 
    return date_time

def getLikesShares(body):
    likes = ''
    shares = ''
    s_class = body.find("p", class_='s')
    if s_class:
        likes = s_class.find_all('b')[0].get_text()
        shares = s_class.find_all('b')[1].get_text()
    return [likes, shares]

def getQuote(body):
    quotes = []
    content = body.find('div', class_='narrow')
    blockquotes = content.find_all('blockquote')
    for blockquote in blockquotes:
        a_tag = blockquote.find('a')
        if a_tag:
            _id = a_tag.get('href')
            quotes.append(_id)
    return quotes

def getText(body):
    content = body.find('div', class_='narrow')
    quotes = getQuote(body)
    text = ''
    for i in range(0, len(quotes)):
        content.blockquote.extract()
    text = content.get_text()
    return text

def getPostID(header):
    post_id = ''
    name = header.find_all('a')[0].get('name')
    if name:
        post_id = name
    return post_id

def is_post_equal(post1, post2):
    return post1 == post2

In [64]:
# retrieve details of each post
def parse_post(header, body):
    post = {}
    post['posted'] = getTimestamp(header)
    post['retrieved'] = datetime.now().strftime("%H:%M:%S %d-%m-%Y")
    post['shares'] = getLikesShares(body)[1]
    post['likes'] = getLikesShares(body)[0]
    post['user'] = getUser(header)
    post['has_quote'] = True if getQuote(body) else False
    post['quotes'] = getQuote(body)
    post['text'] = getText(body)
    post['post_id'] = getPostID(header)
    return post

In [65]:
# retrieve posts from thread
def parse_thread(thread):
    page = 0 # start from the first page
    next_page = True
    index_post = ''
    previous_index_post = ''

    data = []

    while next_page:
        start_url = 'https://www.nairaland.com/{}/{}'.format(thread, page)
        r1 = requests.get(start_url, heads)
        thread_html = BeautifulSoup(r1.text, 'lxml')

        headers = thread_html.find_all('td', class_='bold l pu')
        bodys = thread_html.find_all('td', class_='l w pd')

        #retrieve first post in the thread 
        index_post = getPostID(headers[0]) 

        if page > 1:
            # compare first post on current page with previous page
            if is_post_equal(index_post, previous_index_post):
                break

        for i in range(len(headers)):
            header = headers[i]
            body = bodys[i]
            post = parse_post(header, body)
            post.update({''})
            data.append(post)
        
        previous_index_post = index_post
        page += 1
    return data

In [66]:
data = parse_thread('5681629')

In [67]:
df = pd.DataFrame(data)

In [68]:
len(df)

64